## Setup

In [ ]:
import sys
from pathlib import Path

from aocd import get_data, submit

In [2]:
# Add parent directory to path to allow relative imports into Jupyter notebook
sys.path.append(str(Path.cwd().parent))

In [91]:
# Get raw advent-of-code data
data: str = get_data(year=2024, day=9)

## Part a

In [33]:
# Constants
FREE_SPACE_MARKER = -1  # -1 is used as a free space marker to avoid type conflicts

In [99]:
# Unpack data
files = [(file_id, int(file_size)) for file_id, file_size in enumerate(data[::2])]
free_space_sizes = [*map(int, data[1::2]), 0]  # Append zero-size free space to ensure equal length to files

# Generate disk blocks
blocks = []
for (file_id, file_size), free_space in zip(files, free_space_sizes, strict=True):
    blocks.extend([file_id] * file_size + [FREE_SPACE_MARKER] * free_space)

In [100]:
# Compact disk
last_file_block_index: int = len(blocks) - 1  # Index of the right-most file block
for i, block in enumerate(blocks):
    # Check if there are no more free spaces to the left of the right-most file block
    if last_file_block_index < i:
        break
    if block == FREE_SPACE_MARKER:
        # Swap right-most file block and free space
        blocks[i], blocks[last_file_block_index] = blocks[last_file_block_index], FREE_SPACE_MARKER

        # Find the new index of the right-most file block
        while blocks[last_file_block_index] == FREE_SPACE_MARKER:
            last_file_block_index -= 1

# Calculate filesystem checksum. No need to check further than the last file block index
checksum_a = sum(position * file_id for position, file_id in enumerate(blocks[: last_file_block_index + 1]))

In [ ]:
# Submit answer
submit(checksum_a, part="a", day=9, year=2024)

## Part b

In [94]:
# Generate disk fragments, which are full files and free spaces with their block sizes
fragments = []
for (file_id, file_size), free_space_size in zip(files, free_space_sizes, strict=True):
    fragments.extend([(file_id, file_size), (FREE_SPACE_MARKER, free_space_size)])

In [ ]:
# Compact files from highest ID to lowest
for file_id, file_size in reversed(files):
    # Find index of the file fragment in the list of fragments
    file_idx = fragments.index((file_id, file_size))

    # Find leftmost suitable free space left of the file fragment
    for free_space_idx, (free_space_id, free_space_size) in enumerate(fragments[:file_idx]):
        if free_space_id == FREE_SPACE_MARKER and free_space_size >= file_size:
            # In the location of the file, insert a free space fragment
            fragments[file_idx] = (FREE_SPACE_MARKER, file_size)

            # If the free space is the same size as the file, simply replace the free space with the file
            if free_space_size == file_size:
                fragments[free_space_idx] = (file_id, file_size)

            # Else, reduce the size of the free space and insert the file fragment
            else:
                fragments[free_space_idx] = (FREE_SPACE_MARKER, free_space_size - file_size)
                fragments.insert(free_space_idx, (file_id, file_size))

            break

In [103]:
# Generate disk blocks from fragments
blocks = [frag_id for frag_id, frag_size in fragments for _ in range(frag_size)]

# Calculate filesystem checksum, nullifying the free space marker (-1)
checksum_b = sum(position * max(file_id, 0) for position, file_id in enumerate(blocks))

In [ ]:
# Submit answer
submit(checksum_b, part="b", day=9, year=2024)